<a href="https://colab.research.google.com/github/RomainD-prog/ProjetParcours/blob/main/DQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Deep QLearing

## Installation des dépendances

In [ ]:
Le code est inspiré de la documentation KERAS sur le breakout : https://keras.io/examples/rl/deep_q_network_breakout/

In [ ]:
!pip install stable_baselines
#Download and install ROMs
!wget -q http://www.atarimania.com/roms/Roms.rar
!pip install -q unrar
!mkdir ./roms_atari
!unrar x Roms.rar ./roms_atari > /dev/null 2>&1
!python -m atari_py.import_roms ./roms_atari > /dev/null 2>&1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 KB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 38.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 31.0 MB/s eta 0:00:00


In [ ]:
from stable_baselines.common.atari_wrappers import make_atari, wrap_deepmind
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from google.colab import drive
import gym.wrappers
# Creating a folder in Google Disk
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


## Chargement de l'environnement GYM

In [ ]:
env = make_atari("BreakoutNoFrameskip-v4")

/usr/local/lib/python3.8/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.8/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [ ]:
obs = np.array(env.reset())
print(obs.shape)
plt.title("environnement brut")
plt.imshow(obs)


# Il ya ici une erreur d'attribut, il faut ouvrir le fichier erreur et modifier randint par integers et relancer le notebook

AttributeError: ignored

In [ ]:
env = make_atari("BreakoutNoFrameskip-v4")
env = wrap_deepmind(env, frame_stack=True, scale=True)
obs = np.array(env.reset())

print(obs.shape)
plt.title("agent observation (4 frames)")
plt.imshow(obs.transpose([0, 2, 1]).reshape([env.observation_space.shape[0], -1]))

In [ ]:

print(env.action_space)
print(env.get_action_meanings())
     

In [ ]:
#Déclaration des paramètres d'états
env.reset()
rewards = []
while True:
    obs, rew, done, info = env.step(env.action_space.sample())
    rewards.append(rew)
    if done:
        break
plt.plot(rewards)
     

## Début du traitement avec création du CNN

In [ ]:

num_actions = 4
def create_q_model():
    # Réseau fortement inspiré du papier Attari sur RL
    inputs = layers.Input(shape=(84, 84, 4,))

    layer1 = layers.Conv2D(32, 8, strides=4, activation="relu")(inputs)
    layer2 = layers.Conv2D(64, 4, strides=2, activation="relu")(layer1)
    layer3 = layers.Conv2D(64, 3, strides=1, activation="relu")(layer2)
    layer4 = layers.Flatten()(layer3)
    layer5 = layers.Dense(512, activation="relu")(layer4)
    action = layers.Dense(num_actions, activation="linear")(layer5)

    return keras.Model(inputs=inputs, outputs=action)

# Le réseau optimise la valeur de Q qui caractérise une fonction
model = create_q_model()
# on affiche le modèle
model_target = create_q_model()
model.summary()

In [ ]:
gamma = 0.99   # Discount factor for past rewards

# On définit nos paramètres epsilon d'exploration
epsilon = 1.0  
epsilon_max_1 = 1.0 
epsilon_min_1 = 0.2  
epsilon_max_2 = epsilon_min_1  
epsilon_min_2 = 0.1
epsilon_max_3 = epsilon_min_2  
epsilon_min_3 = 0.02

epsilon_interval_1 = (epsilon_max_1 - epsilon_min_1)  
epsilon_interval_2 = (epsilon_max_2 - epsilon_min_2)  
epsilon_interval_3 = (epsilon_max_3 - epsilon_min_3)  

# Nombre maximum de frame
epsilon_greedy_frames = 1000000.0

# Numbre de frame dans lesquels on effectue une action aléatoire
epsilon_random_frames = 50000

# Mémoire de saturation=> dépend de la capacité de notre machine
max_memory_length = 1900

# Taille du batch 
#nombre max d'étapes par épisode sinon il se peut que le paddle ne touche 
#la boule. Cela permet d'éviter que l'entrainement tourne en boucle 
batch_size = 32  
max_steps_per_episode = 10000

# toutes les 20 actions, on entraine le modèle
update_after_actions = 20

# on actualise le
update_target_network = 10000

# Dans l'article de Deepmind, ils utilisent RMSProp, mais l'optimiseur Adam améliore le temps de formation.
optimizer = keras.optimizers.Adam(learning_rate=0.00025, clipnorm=1.0)

# Using huber loss for stability
loss_function = keras.losses.Huber()

In [ ]:
# On initialise les listes pour stocker les résultats
action_history = []
state_history = []
state_next_history = []
rewards_history = []
done_history = []

episode_reward_history = []
running_reward = 0
episode_count = 0
frame_count = 0

while True:  # On tourne dans la boucle tant que l'agent n'a pas résolu le problème
    state = np.array(env.reset())
    episode_reward = 0

    for timestep in range(1, max_steps_per_episode):
        
        frame_count += 1

        # On utilise la méthode d'exploration epsilon greedy
        if frame_count < epsilon_random_frames or epsilon > np.random.rand(1)[0]:
            # Take random action
            action = np.random.choice(num_actions)
        else:
            # On prédit la valeur de Q 
            # A chaque état au sein de l'environnement
            state_tensor = tf.convert_to_tensor(state)
            state_tensor = tf.expand_dims(state_tensor, 0)
            action_probs = model(state_tensor, training=False)
            # Take best action
            action = tf.argmax(action_probs[0]).numpy()

        # Decay probability of taking random action
        if frame_count < epsilon_greedy_frames:
          epsilon -= epsilon_interval_1 / epsilon_greedy_frames
          epsilon = max(epsilon, epsilon_min_1)
        
        if frame_count > epsilon_greedy_frames and frame_count < 2 * epsilon_greedy_frames:
          epsilon -= epsilon_interval_2 / epsilon_greedy_frames
          epsilon = max(epsilon, epsilon_min_2)
        
        if frame_count > 2 * epsilon_greedy_frames:
          epsilon -= epsilon_interval_3 / epsilon_greedy_frames
          epsilon = max(epsilon, epsilon_min_3)
          

        # On applique les actions filtrées à l'environnement
        state_next, reward, done, _ = env.step(action)
        state_next = np.array(state_next)

        episode_reward += reward

        # On sauvegarde toute nos actions en mémoire 
        action_history.append(action)
        state_history.append(state)
        state_next_history.append(state_next)
        done_history.append(done)
        rewards_history.append(reward)
        state = state_next

        # Mise à jour toutes les 20 images et dès que la taille du batch est supérieure à 32.
        if frame_count % update_after_actions == 0 and len(done_history) > batch_size:

            # Obtenir les indices des échantillons
            indices = np.random.choice(range(len(done_history)), size=batch_size)

            # On recupère les informations dans les listes concernant les échantillon des états/action/reward
            state_sample = np.array([state_history[i] for i in indices])
            state_next_sample = np.array([state_next_history[i] for i in indices])
            rewards_sample = [rewards_history[i] for i in indices]
            action_sample = [action_history[i] for i in indices]
            done_sample = tf.convert_to_tensor(
                [float(done_history[i]) for i in indices]
            )

            #Construire les valeurs Q mises à jour pour les états futurs échantillonnés.
            #Utiliser le modèle cible pour la stabilité
            future_rewards = model_target.predict(state_next_sample)
            # Q value = reward + discount factor * expected future reward
            updated_q_values = rewards_sample + gamma * tf.reduce_max(
                future_rewards, axis=1
            )

            # Si c'est la dernière image, la dernière valeur sera -1
            updated_q_values = updated_q_values * (1 - done_sample) - done_sample

            #on crée un masque pour que nous calculer la loss que sur les valeurs Q mises à jour.
            masks = tf.one_hot(action_sample, num_actions)

            with tf.GradientTape() as tape:
                # On entraine le modèle sur les états et les Q values actualisées
                q_values = model(state_sample)

                # on applique les masques aux valeurs Q pour obtenir la valeur Q de l'action entreprise.
                q_action = tf.reduce_sum(tf.multiply(q_values, masks), axis=1)
                # on recalcule une loss entre l'ancienne et la nouvelle q value
                loss = loss_function(updated_q_values, q_action)

            # Backpropagation
            grads = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(grads, model.trainable_variables))

        if frame_count % update_target_network == 0:
            #on actualise avec les ouveaux poids
            model_target.set_weights(model.get_weights())
            # on décrit lors des étapes
            template = "running reward: {:.2f} at episode {}, frame count {}, epsilon {:.3f}, loss {:.5f}"
            print(template.format(running_reward, episode_count, frame_count, epsilon, loss))

        #on limite l'historique des rewards et des états
        if len(rewards_history) > max_memory_length:
            del rewards_history[:1]
            del state_history[:1]
            del state_next_history[:1]
            del action_history[:1]
            del done_history[:1]

        if done:
            break

    episode_reward_history.append(episode_reward)
    if len(episode_reward_history) > 100:
        del episode_reward_history[:1]
    running_reward = np.mean(episode_reward_history)

    episode_count += 1

    if running_reward > 5:  # On considère ici que l'agent a resolu le jeu dès que sa reward dépasse 5
        print("Solved at episode {}!".format(episode_count))
        break

In [ ]:
# Sauvegarde du modèle
#model_name = 'breakout_model_1'
path = F"/content/gdrive/MyDrive/3A/Projet Parcours/models/{model_name}" 
model.save(path)

# Chargement du modèle
model = tf.keras.models.load_model(path)

#model_name = 'breakout'
#path = F"/content/gdrive/MyDrive/{model_name}" 

#model = tf.keras.models.load_model(path)


In [ ]:
def make_env():
  env = make_atari("BreakoutNoFrameskip-v4")
  env = wrap_deepmind(env, frame_stack=True, scale=True)
  return env
     

In [ ]:
from gym.wrappers.monitoring.video_recorder import VideoRecorder

In [ ]:
# On paramètre l'affichage et le recording de notre breakout sur la base du modèle qu'on vient d'entrainer

env = make_env()
env = gym.wrappers.RecordVideo(env, "/content/gdrive/MyDrive/3A/Projet Parcours/models/breakout_model_1/vid1")

observation = env.reset()
info = 0
reward_window = []
reward_signal_history = []
epsilon_history = []

hits = []
bltd = 3 #On indique le nombre de brique l'on veut voir détruites

for i_episode in range(1):
    reward_window=[] 
    epsilon = 0  
    for t in range(100):# on parcourt le nombre de frames correspondantes 
        
        if epsilon > np.random.rand(1)[0]:
          action = np.random.choice(num_actions)
        else:
          state_tensor = tf.convert_to_tensor(observation)
          state_tensor = tf.expand_dims(state_tensor, 0)
          action_probs = model(state_tensor, training=False)
          action = tf.argmax(action_probs[0]).numpy()
        
        observation, reward, done, info = env.step(action)
        hits.append(reward)
        reward_window.append(reward)
        if len(reward_window) > 200:
          del reward_window[:1] 
        if len(reward_window) == 200 and np.sum(reward_window) == 0:
          epsilon = 0.01
        else:
          epsilon = 0.0001

        epsilon_history.append(epsilon)
        reward_signal_history.append(reward)

        
        if done:
            print("Lost one life after {} timesteps".format(t+1))
            print(info)
            # On plot l'évolution d'epsilon et des rewards
            fig,ax=plt.subplots(figsize=(20,3))
            #plt.clf()
            ax.plot(epsilon_history, color="red")
            ax.set_ylabel("epsilon",color="red",fontsize=14)
            ax2=ax.twinx()
            ax2.plot(reward_signal_history,color="blue")
            ax2.set_ylabel("reward_signal",color="blue",fontsize=14)
            plt.show()

            epsilon_history = []
            reward_signal_history = []
            
            bltd = bltd-np.sum(hits)
            hits = []
            print("Bricks left to destroy ", bltd)
            if info['ale.lives'] == 0:
              break

            env.reset()
env.close()